In [25]:
import os
os.environ['TESSDATA_PREFIX'] = '/opt/homebrew/share/tessdata'

In [30]:
#Install dependencies (run once per environment)
%pip install pytesseract opencv-python pdf2image Pillow


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [44]:


# --- Imports ---
import pytesseract
from pdf2image import convert_from_path
import cv2
from PIL import Image
import os
from tqdm import tqdm  # optional, for progress bar

# --- Paths ---
pdf_path = "../PDFs/tbmm17004078.pdf"    # your input PDF
output_dir = "outputocr"              # folder to save output
os.makedirs(output_dir, exist_ok=True)

# --- Convert PDF pages to high-resolution images ---
print("Converting PDF to images...")
pages = convert_from_path(pdf_path, dpi=300)

# --- OCR Loop ---
all_text = ""

for i, page in enumerate(tqdm(pages, desc="Processing pages")):
    image_path = os.path.join(output_dir, f"page_{i+1}.png")
    page.save(image_path, "PNG")

    # --- Read image for preprocessing ---
    img = cv2.imread(image_path)

    # Preprocessing steps for old documents
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # grayscale
    gray = cv2.GaussianBlur(gray, (3, 3), 0)      # smooth noise
    gray = cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        31, 2
    )                                             # binarize & enhance contrast
    gray = cv2.fastNlMeansDenoising(gray, None, 30, 7, 21)  # denoise

    # --- Run Tesseract OCR ---
    text = pytesseract.image_to_string(
        gray,
        lang="tur",
        config="--oem 1 --psm 3"
    )


    # --- Add to combined output ---
    all_text += f"\n\n===== PAGE {i+1} =====\n\n" + text

# --- Save combined text file ---
combined_path = os.path.join(output_dir, "combined_output_psm3_tesseract.txt")
with open(combined_path, "w", encoding="utf-8") as f:
    f.write(all_text)

print("✅ OCR complete! All pages processed.")
print(f"Combined text file saved at: {combined_path}")






Converting PDF to images...


Processing pages: 100%|██████████| 52/52 [03:29<00:00,  4.03s/it]

✅ OCR complete! All pages processed.
Combined text file saved at: outputocr/combined_output_psm3_tesseract.txt


In [41]:
# ## OPTION2 : USE unstructured-ocr package
%pip install "unstructured[local-inference,pdf,ocr]" poppler-utils   pdfminer.six  onnxruntime==1.20.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 4.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: onnxruntime
    Found existing installation: onnxruntime 1.23.1
    Uninstalling onnxruntime-1.23.1:
      Successfully uninstalled onnxruntime-1.23.1

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [42]:
output_path = "outputocr/unstructured_ocr_output.txt"

from unstructured.partition.pdf import partition

elements = partition(filename=pdf_path, languages=["tur"])
with open(output_path, "w", encoding="utf-8") as f:
    for element in elements:
        f.write(element.text + "\n")

ImportError: cannot import name 'cuda_version' from 'onnxruntime.capi.onnxruntime_validation' (/Users/wbagger/Documents/Semantic-Analysis-of-TBMM-Transcripts/myenv/lib/python3.13/site-packages/onnxruntime/capi/onnxruntime_validation.py)